# Notebook 4: Trading Live (Simulation et Supervision)

## Objectif
Ce notebook permet de démarrer le pipeline de trading live (initialement en mode dry-run ou paper trading), de configurer les paramètres essentiels, de superviser son état en temps réel, et d'illustrer comment des interactions (comme des overrides via Telegram) pourraient fonctionner. 

**Avertissement:** Ce notebook est destiné à l'expérimentation et à la supervision. Pour une exécution de trading live en production continue, un service dédié (comme celui géré par `systemd`) est recommandé.

## 1. Configuration Globale du Notebook

Modifiez les variables dans la cellule suivante pour configurer le notebook, y compris les clés API si elles ne sont pas définies comme variables d'environnement.

In [ ]:
# --- Initialisation des Modules Python (Obligatoire en premier) ---
import asyncio
import json
import pandas as pd
import sys
import os
import time
import logging
import matplotlib.pyplot as plt
import seaborn as sns

# --- Booléens pour activer/désactiver des sections ---
RUN_LIVE_PIPELINE_EXECUTION = False # True pour lancer réellement le pipeline live (ATTENTION)
PARAM_DRY_RUN = True               # True: pas d'ordres réels. False: interagit avec l'exchange (Testnet recommandé!)
SIMULATE_OVERRIDE = False          # True pour tester la simulation d'une commande d'override

# --- Configuration des Clés API (IMPORTANT: SÉCURITÉ) ---
API_KEYS_INPUT_LIVE = {
    "BINANCE_API_KEY": "VOTRE_BINANCE_KEY_ICI_SI_NON_DEFINIE_EN_ENV",
    "BINANCE_API_SECRET": "VOTRE_BINANCE_SECRET_ICI_SI_NON_DEFINI_EN_ENV",
    "BITGET_API_KEY": "VOTRE_BITGET_KEY_ICI_SI_NON_DEFINIE_EN_ENV",
    "BITGET_API_SECRET": "VOTRE_BITGET_SECRET_ICI_SI_NON_DEFINI_EN_ENV",
    "BITGET_PASSPHRASE": "VOTRE_BITGET_PASSPHRASE_ICI_SI_NON_DEFINI_EN_ENV",
    "TELEGRAM_BOT_TOKEN": "VOTRE_TELEGRAM_TOKEN_ICI_SI_NON_DEFINI_EN_ENV",
    "TELEGRAM_CHAT_ID": "VOTRE_TELEGRAM_CHAT_ID_ICI_SI_NON_DEFINI_EN_ENV"
}

# --- Configuration des Logs et Graphiques ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
sns.set_theme(style="whitegrid"); plt.rcParams['figure.figsize'] = (18, 5); plt.rcParams['figure.dpi'] = 100

# --- Configuration du PYTHONPATH et Import des Modules Projet ---
PROJECT_ROOT_NOTEBOOK_LEVEL = os.path.abspath(os.getcwd())
PROJECT_CODE_ROOT = os.path.join(PROJECT_ROOT_NOTEBOOK_LEVEL, "ultimate")
if PROJECT_CODE_ROOT not in sys.path: sys.path.append(PROJECT_CODE_ROOT); logger.info(f"'{PROJECT_CODE_ROOT}' ajouté au PYTHONPATH.")

cfg_instance = None; run_pipeline_main = None # Placeholders
try:
    from run_enhanced_pipeline import main as run_pipeline_main_imported
    run_pipeline_main = run_pipeline_main_imported
    from config.config import Config
    logger.info("Modules projet importés.")
    cfg_instance = Config() 
    logger.info(f"Instance de Config créée. Project root utilisé par Config: {getattr(cfg_instance, '_project_root', 'Non défini par la classe Config')}")
    if not cfg_instance.yaml_config: 
        logger.error("CRITIQUE: config.yaml non chargé par Config(). Assurez-vous qu'il est dans 'Morningstar/config/config.yaml' et que la classe Config le trouve.")
except ImportError as e: 
    logger.error(f"Erreur import modules projet: {e}. Certaines fonctionnalités seront désactivées.", exc_info=True)
    if cfg_instance is None: 
        class FallbackConfig: 
            _project_root = PROJECT_ROOT_NOTEBOOK_LEVEL
            yaml_config = {}
            def get_config(self, key, default=None): return default
        cfg_instance = FallbackConfig()
        logger.warning("Utilisation d'une instance Config de fallback suite à une erreur d'import.")
except Exception as e: 
    logger.error(f"Erreur majeure lors de l'initialisation de Config: {e}. Utilisation de FallbackConfig.", exc_info=True)
    if cfg_instance is None:
        class FallbackConfig: 
            _project_root = PROJECT_ROOT_NOTEBOOK_LEVEL
            yaml_config = {}
            def get_config(self, key, default=None): return default
        cfg_instance = FallbackConfig()
        logger.warning("Utilisation d'une instance Config de fallback suite à une erreur majeure.")

# --- Définition des Clés API Saisies (si non présentes dans l'env) ---
for key, value in API_KEYS_INPUT_LIVE.items():
    env_value = os.getenv(key)
    if "_ICI_SI_NON_DEFINI_EN_ENV" not in value and value.strip() != "": 
        os.environ[key] = value; 
        if env_value != value : logger.info(f"Variable d'env '{key}' définie/surchargée pour cette session.")
    elif not env_value: 
        logger.warning(f"Clé API '{key}' non définie (ni via env, ni via saisie directe).")
    else: 
        logger.info(f"Clé API '{key}' utilisée depuis l'environnement.")

# --- Définition des Chemins ---
STATUS_FILE_PATH = os.path.join(PROJECT_CODE_ROOT, cfg_instance.get_config("paths.live_status_file", "live_status.json").lstrip("./\\ "))
OVERRIDE_COMMAND_FILE = os.path.join(PROJECT_CODE_ROOT, cfg_instance.get_config("paths.override_command_file", "data/ipc/override_command.json").lstrip("./\\ "))
logger.info(f"Fichier de statut live attendu: {STATUS_FILE_PATH}")
logger.info(f"Fichier de commande d'override attendu: {OVERRIDE_COMMAND_FILE}")

## 2. Vérification de la Configuration pour le Trading Live

Cette section vérifie les paramètres essentiels lus depuis `config.yaml` et les variables d'environnement pour les clés API.

In [ ]:
logger.info("--- Vérification de la Configuration pour le Live Trading ---")
can_run_live_safely = True
configured_symbols = [] # Initialisation

exchange_id_from_config = cfg_instance.get_config("exchange.id", "binance").upper()
required_keys_for_exchange = [f"{exchange_id_from_config}_API_KEY", f"{exchange_id_from_config}_API_SECRET"]
if exchange_id_from_config in ["KUCOIN", "BITGET"]: required_keys_for_exchange.append(f"{exchange_id_from_config}_PASSPHRASE")
all_exchange_keys_found = all(os.getenv(key) and "_ICI_SI_NON_DEFINI_EN_ENV" not in os.getenv(key, "") for key in required_keys_for_exchange)

if all_exchange_keys_found:
    logger.info(f"Clés API pour l'exchange '{exchange_id_from_config}' DÉFINIES.")
else:
    logger.warning(f"ATTENTION: Clés API pour '{exchange_id_from_config}' MANQUANTES ou non remplacées.")
    for key_name in required_keys_for_exchange: 
        val = os.getenv(key_name, API_KEYS_INPUT_LIVE.get(key_name, ""))
        if not val or "_ICI_SI_NON_DEFINI_EN_ENV" in val: logger.warning(f"  - Clé manquante/non remplacée: {key_name}")
    if not PARAM_DRY_RUN: can_run_live_safely = False

telegram_enabled = cfg_instance.get_config("telegram.enabled", False)
if telegram_enabled:
    tg_token_val = os.getenv("TELEGRAM_BOT_TOKEN", API_KEYS_INPUT_LIVE.get("TELEGRAM_BOT_TOKEN", ""))
    tg_chat_id_val = os.getenv("TELEGRAM_CHAT_ID", API_KEYS_INPUT_LIVE.get("TELEGRAM_CHAT_ID", ""))
    if tg_token_val and "_ICI_SI_NON_DEFINI_EN_ENV" not in tg_token_val and tg_chat_id_val and "_ICI_SI_NON_DEFINI_EN_ENV" not in tg_chat_id_val:
        logger.info("Notifications Telegram activées et clés API DÉFINIES.")
    else: logger.warning("Notifications Telegram activées, mais clés MANQUANTES/non remplacées.")
else: logger.info("Notifications Telegram désactivées dans config.yaml.")

configured_symbols = cfg_instance.get_config('exchange.symbols', [])
if isinstance(configured_symbols, list) and configured_symbols:
    logger.info(f"Paires à trader (config.yaml -> exchange.symbols): {', '.join(configured_symbols)}")
else:
    logger.error("ERREUR CRITIQUE: 'exchange.symbols' non défini ou n'est pas une liste valide dans config.yaml.")
    can_run_live_safely = False

logger.info(f"Paramètres de cette session: Mode Live={PARAM_LIVE_MODE_ENABLED}, Dry Run={PARAM_DRY_RUN}")
if not PARAM_DRY_RUN and not all_exchange_keys_found:
    logger.error("ERREUR CRITIQUE: PARAM_DRY_RUN est False mais les clés API de l'exchange ne sont pas toutes définies.")
    can_run_live_safely = False

if not can_run_live_safely and RUN_LIVE_PIPELINE_EXECUTION and not PARAM_DRY_RUN:
    logger.error("Le pipeline live ne sera pas lancé en mode réel à cause des erreurs de configuration ci-dessus.")
    RUN_LIVE_PIPELINE_EXECUTION = False

## 3. Lancement du Pipeline Live

Appelle `ultimate/run_enhanced_pipeline.py`.
**Attention :** Si `RUN_LIVE_PIPELINE_EXECUTION` est `True`, cela démarrera une boucle de trading.

In [ ]:
if RUN_LIVE_PIPELINE_EXECUTION:
    if not can_run_live_safely and not PARAM_DRY_RUN:
        logger.error("Lancement du pipeline live en mode réel annulé (erreurs de configuration).")
    elif not (isinstance(configured_symbols, list) and configured_symbols):
        logger.error("Lancement annulé: 'exchange.symbols' non configuré.")
    else:
        logger.info("\nPréparation au lancement du pipeline en mode live...")
        cli_args = []
        if PARAM_LIVE_MODE_ENABLED: cli_args.append('--live')
        if PARAM_DRY_RUN: cli_args.append('--dry-run')

        original_argv_live = sys.argv
        script_path_live = os.path.join(PROJECT_CODE_ROOT, 'run_enhanced_pipeline.py')
        sys.argv = [script_path_live] + cli_args

        logger.info(f"Appel de {script_path_live} avec args: {sys.argv[1:]}")
        print("-"*80 + "\nLANCEMENT DU PIPELINE LIVE... Interrompez le noyau pour arrêter.\n" + "-"*80)
        try:
            if run_pipeline_main is not None and callable(run_pipeline_main):
                run_pipeline_main()
            else: logger.error("`run_pipeline_main` non importé/callable. Lancement annulé.")
        except KeyboardInterrupt: logger.info("Pipeline live interrompu par l'utilisateur.")
        except SystemExit: logger.info("Pipeline live interrompu (possiblement par argparse).")
        except Exception as e: logger.error(f"Erreur majeure pipeline live: {e}", exc_info=True)
        finally: sys.argv = original_argv_live; logger.info("Arguments sys.argv restaurés.")
else:
    logger.info("Lancement du pipeline live désactivé (RUN_LIVE_PIPELINE_EXECUTION=False).")

## 4. Supervision & Logs (Après exécution du pipeline)

Inspecte `ultimate/live_status.json` et visualise l'évolution du PnL.

In [ ]:
def display_live_status(status_file_path):
    if os.path.exists(status_file_path):
        try:
            with open(status_file_path, 'r') as f: status = json.load(f)
            logger.info(f"\n--- Statut Live Actuel (depuis {status_file_path}) ---")
            print(json.dumps(status, indent=2))
            
            last_update_dt = pd.to_datetime(status.get('timestamp'), unit='s', utc=True).tz_convert('Europe/Paris') if status.get('timestamp') else 'N/A'
            print(f"\n**Dernière Mise à Jour:** {last_update_dt}")
            print(f"**Trading Actif:** {status.get('trading_active', False)}")
            print(f"**Mode Dry Run:** {status.get('dry_run', 'N/A')}")
            current_symbols_trading = status.get('symbols', [])
            print(f"**Paires surveillées:** {current_symbols_trading if current_symbols_trading else ['N/A']}") 
            
            if isinstance(status.get('portfolio_status'), dict):
                for symbol, sym_status in status['portfolio_status'].items():
                    print(f"  **Statut pour {symbol}:**")
                    print(f"    Position: {sym_status.get('position_side', 'FLAT')} (Taille: {sym_status.get('current_position_size', 0)})")
                    print(f"    Prix d'entrée: {sym_status.get('entry_price', 0):.4f}")
                    print(f"    PnL non réalisé: {sym_status.get('unrealized_pnl', 0):.2f}")
            print(f"**PnL Cumulé Total:** {status.get('cumulative_pnl_total', 0):.2f}")
            print(f"**Dernier Signal Global:** {status.get('last_signal_type', 'N/A')} pour {status.get('last_signal_symbol', 'N/A')} à {status.get('last_signal_price', 'N/A')}")
            
            if 'pnl_history' in status and isinstance(status['pnl_history'], list) and len(status['pnl_history']) > 0:
                pnl_hist_df = pd.DataFrame(status['pnl_history'])
                if 'timestamp' in pnl_hist_df.columns and 'cumulative_pnl' in pnl_hist_df.columns:
                    pnl_hist_df['timestamp'] = pd.to_datetime(pnl_hist_df['timestamp'], unit='s')
                    plt.figure(figsize=(18,6))
                    plt.plot(pnl_hist_df['timestamp'], pnl_hist_df['cumulative_pnl'], marker='.', linestyle='-', label='PnL Cumulé')
                    plt.title('Évolution du PnL Cumulé (depuis live_status.json)'); plt.xlabel('Timestamp'); plt.ylabel('PnL Cumulé')
                    plt.legend(); plt.grid(True); plt.tight_layout(); plt.show()
                else: logger.warning("Colonnes 'timestamp' ou 'cumulative_pnl' manquantes dans pnl_history.")
            else: logger.info("Pas d'historique de PnL à visualiser dans live_status.json.")
        except json.JSONDecodeError: logger.error(f"Erreur décodage JSON: {status_file_path}")
        except Exception as e: logger.error(f"Erreur lecture/affichage statut: {e}", exc_info=True)
    else: logger.warning(f"\nFichier statut {STATUS_FILE_PATH} non trouvé.")

logger.info(f"Tentative d'affichage du statut depuis {STATUS_FILE_PATH}...")
display_live_status(STATUS_FILE_PATH)
print("\nPour une supervision en temps réel, exécutez cette cellule périodiquement.")

## 5. Interaction avec le Bot Telegram (Simulation d'Override)

Si le bot Telegram est actif et communique avec `LiveExecutor` (ex: via `ultimate/data/ipc/override_command.json`), une commande `/override` pourrait influencer `LiveExecutor`.

In [ ]:
logger.info(f"Le LiveExecutor pourrait écouter les commandes d'override dans: {OVERRIDE_COMMAND_FILE}")

def simulate_telegram_override(action: str, symbol: str, price: float = None, amount_quote: float = None, command_file: str = OVERRIDE_COMMAND_FILE):
    command = {
        "timestamp": pd.Timestamp.now(tz='UTC').isoformat(),
        "action": action.upper(), "symbol": symbol.upper(),
    }
    if price is not None: command["price"] = price; command["type"] = "LIMIT" 
    else: command["type"] = "MARKET"
    if amount_quote is not None: command["amount_quote"] = amount_quote 
        
    logger.info(f"Simulation écriture commande override dans {command_file}: {command}")
    try:
        os.makedirs(os.path.dirname(command_file), exist_ok=True)
        with open(command_file, 'w') as f: json.dump(command, f, indent=2)
        logger.info("Commande override simulée écrite. Vérifiez logs LiveExecutor et live_status.json.")
    except Exception as e: logger.error(f"Erreur simulation écriture commande override: {e}")

if SIMULATE_OVERRIDE:
    if configured_symbols and isinstance(configured_symbols, list) and len(configured_symbols) > 0:
       selected_pair_for_override = configured_symbols[0]
       logger.info(f"Simulation d'un override d'ACHAT MARKET pour {selected_pair_for_override}...")
       simulate_telegram_override("BUY", selected_pair_for_override, amount_quote=100) # Acheter pour 100 USDT par exemple
       logger.info("Attente de 15s pour réaction LiveExecutor..."); time.sleep(15) 
       display_live_status(STATUS_FILE_PATH) 
    else: 
       logger.warning("Aucune paire configurée dans config.yaml (exchange.symbols) pour simuler un override.")
else:
    logger.info("Simulation d'override désactivée (SIMULATE_OVERRIDE=False).")

print("\nPour un vrai override, utilisez la commande /override du bot Telegram si celui-ci est actif et connecté au pipeline live.")